In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install pytorch-tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.drop(columns=['user_id'], inplace=True)
df['geo_id'] = df["geo_id"].astype("object")
df = df[df['referer'].apply(lambda x: isinstance(x, str))]

In [4]:
def split_referer(referer):
    referer = referer[8::]
    parts = referer.split('/', 2)
    domain = parts[0]
    if len(parts) > 1 and parts [1]:
        path = parts[1]
    else:
        path = 'nopath'
    return domain, path

In [5]:
df[['domain', 'path']] = df['referer'].apply(split_referer).tolist()

In [6]:
df.drop(columns=['referer'], inplace=True)

In [7]:
categorical_features = ['country_id', 'geo_id', 'region_id', 'timezone_id', 'browser', 'browser_version', 'os', 'os_version', 'domain', 'path']
features_to_scale = ['request_ts', 'component0', 'component1', 'component2', 'component3', 'component4', 'component5', 'component6', 'component7', 'component8', 'component9']

In [ ]:
df = pd.get_dummies(df, columns=categorical_features, prefix=categorical_features)

In [8]:
scaler = MinMaxScaler()
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])

In [10]:
X = df.drop('target', axis=1)
y = df['target']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=42)

In [11]:
X_train = X_train.values
X_valid = X_valid.values
X_test = X_test.values
y_train = y_train.values
y_valid = y_valid.values
y_test = y_test.values

In [ ]:
clf = TabNetClassifier()
clf.fit(
  X_train, y_train,
  eval_set=[(X_valid, y_valid)],
  max_epochs=100
)

In [ ]:
preds = clf.predict(X_test)